# Fr&#233;chet Derivative and Matrix Calculus
This article introduces Fr&#233;chet derivative, a perspective towards derivatives and gradients that enables efficient computation of gradients w.r.t. matrices which is especially useful for deriving back propagation rules for neural networks in matrix form. This article then takes multilayer perceptron and batch normalization as examples and derives the gradients in matrix form, followed by corresponding code implementations.

## From Limits to Fr&#233;chet Derivative
For real-valued function $f: \mathbb{R} \to \mathbb{R}$, if the derivative at point $x$ exists, it is defined as

$$
\begin{align*}
    f'(x) = \lim_{\epsilon \to 0}\frac{f(x + \epsilon) - f(x)}{\epsilon}.
\end{align*}
$$

While this definition generalizes to vector- and matrix-valued functions, and is often related with the slope of $f$ at some point of interest, the concept of derivative or gradient could be viewed from another perspective, which is to approximate the difference in function value with a term that is linear w.r.t. the difference in the variable:

$$
\begin{align*}
    f(x + \epsilon) = f(x) + f'(x)\epsilon + o(\epsilon)
\end{align*}
$$

where $o(\epsilon)$ is a term that shrinks much faster than $\epsilon$. Technically, it refers to some term $g(\epsilon)$ that satisfies

$$
\begin{align*}
    \lim_{\epsilon \to 0}\frac{g(\epsilon)}{\epsilon} = 0.
\end{align*}
$$

This view of derivative is called the Fr&#233;chet derivative. The implication of which on finding the derivative at a point is that if we could manage to separate a linear term from the difference in the function value while the residual is of $o(\epsilon)$, then the separated weights is exactly the derivative that we are looking for. Now that we've grasped the idea of the Fr&#233;chet derivative by looking at real-valued functions, we'll move on to see it's generalized form to vector- and matrix-valued functions, which is achieved via the inner product.

For vector-valued function $f: \mathbb{R}^n \to \mathbb{R}$, the linear term is computed as the vector inner-product between the gradient and the differences in the variables. Each variable is assigned with a partial derivative, and the differences in all variables are aggregated to get the approximation:

$$
\begin{align*}
    f(\mathbf{x} + \boldsymbol\epsilon)
    &= f(\mathbf{x})
        + \langle{\nabla_\mathbf{x}(f), \boldsymbol\epsilon}\rangle
        + o(\lVert{\boldsymbol\epsilon}\rVert) \\
    &= f(\mathbf{x})
        + {\nabla_\mathbf{x}(f)}^\top\boldsymbol\epsilon
        + o(\lVert{\boldsymbol\epsilon}\rVert).
\end{align*}
$$

For matrix-valued function $f: \mathbb{R}^{m \times n} \to \mathbb{R}$, the linear term is computed as the matrix inner-product, which equals to the vector inner product after we vectorize the two matrices, or the sum of the elements after a Hadamard product, but most importantly, the trace of the matrix computed by multiplying the two matrices with one of them transposed. The idea here is essentially the same as for vector-valued functions in terms that we aggregate the differences in all variables:

$$
\begin{align*}
    f(\mathbf{X} + \boldsymbol{\mathcal{E}})
    &= f(\mathbf{X})
        + \langle{\nabla_\mathbf{X}(f), \boldsymbol{\mathcal{E}}}\rangle
        + o(\lVert{\boldsymbol{\mathcal{E}}}\rVert_F) \\
    &= f(\mathbf{X})
        + \langle{\operatorname{vec}(\nabla_\mathbf{X}(f)), \operatorname{vec}(\boldsymbol{\mathcal{E}})}\rangle
        + o(\lVert{\boldsymbol{\mathcal{E}}}\rVert_F) \\
    &= f(\mathbf{X})
        + \mathbf{1}^\top\cdot(\nabla_\mathbf{X}(f)\odot\boldsymbol{\mathcal{E}})\cdot\mathbf{1}
        + o(\lVert{\boldsymbol{\mathcal{E}}}\rVert_F) \\
    &= f(\mathbf{X})
        + \operatorname{tr}({\nabla_\mathbf{X}(f)}^\top\boldsymbol{\mathcal{E}})
        + o(\lVert{\boldsymbol{\mathcal{E}}}\rVert_F)
\end{align*}
$$

where $\lVert{\boldsymbol{\mathcal{E}}}\rVert_F = \sqrt{\sum_{i=1}^{m}\sum_{j=1}^{n}\mathbf{A}_{i,j}^2} = \sqrt{\operatorname{tr}({\mathbf{A}^\top\mathbf{A}}})$ is the Frobenius norm. The reason that trace is preferred here is because it offers a range of convenient manipulations which are useful to separate out the difference part to get the gradient. We'll see how it works out in the examples. But before that, we need some manipulations of the differential form to properly attribute the difference in loss function to the variables that we would like to compute gradients w.r.t.

## Important Manipulations
### Manipulations of the Differential Form

$$
\begin{align*}
    d(\mathbf{X}\mathbf{Y}) = d(\mathbf{X})\mathbf{Y} + \mathbf{X}d(\mathbf{Y})
\end{align*}
$$

### Manipulations of the $\textit{trace}$ Operation

$$
\begin{align}
    \operatorname{tr}(\mathbf{X}\mathbf{Y}) = \operatorname{tr}(\mathbf{Y}\mathbf{X})
\end{align}
$$

$Proof$.

$$
\begin{align*}
    \text{LHS}
    = \sum_i\sum_j\mathbf{X}_{i,j}(\mathbf{Y}_{j,i}\mathbf{Z}_{j,i})
    = \sum_i\sum_j(\mathbf{X}_{i,j}\mathbf{Y}^\top_{i,j})\mathbf{Z}_{j,i}
    = \text{RHS}
\end{align*}
$$

$$
\begin{align*}
    \operatorname{tr}(\mathbf{X}(\mathbf{Y}\odot\mathbf{Z})) = \operatorname{tr}((\mathbf{X}\odot\mathbf{Y}^\top)\mathbf{Z})
\end{align*}
$$

$Proof$.

$$
\begin{align*}
    \text{LHS}
    = \sum_i\sum_j\mathbf{X}_{i,j}(\mathbf{Y}_{j,i}\mathbf{Z}_{j,i})
    = \sum_i\sum_j(\mathbf{X}_{i,j}\mathbf{Y}^\top_{i,j})\mathbf{Z}_{j,i}
    = \text{RHS}
\end{align*}
$$

Since neural networks consists of iterating linearity and non-linearity, in backpropagation, $\mathbf{X}$ would be the upstream gradient, $\mathbf{Y}$ would be the gradient of the activation function (since it is applied element-wise), and $\mathbf{Z}$ would be a differential form that entails the variable of interest. This manipulation allows for separating the differential form from the Hadamard product into a multiplicative term.

## Examples
### Multilayer Perceptron
At the $l$-th layer of a multilayer perceptron, the latent representations $\mathbf{H}^{(l - 1)} \in \mathbb{R}^{N \times d^{(l - 1)}}$ from the $(l - 1)$-th layer pass through a linear transformation $\mathbf{W}^{(l)} \in \mathbb{R}^{d^{(l - 1)} \times d^{(l)}}$, followed by a nonlinear activation function $\phi: \mathbb{R} \to \mathbb{R}$ which is applied element-wise, to get the updated representations $\mathbf{H}^{(l)} \in \mathbb{R}^{N \times d^{(l)}}$. We omit the bias term here since it could be easily incorporated by appending a column vector of $1$s to $\mathbf{H}^{(l - 1)}$. Therefore, The forward computation is

$$
\begin{align*}
    \mathbf{H}^{(l)} = \phi(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)}).
\end{align*}
$$

Suppose we've already computed the upstream gradient $\nabla_{\mathbf{H}^{(l)}}(\mathcal{J})$. To proceed on backpropagation with the chain rule we should now compute the local gradient $\nabla_{\mathbf{H}^{(l)}}(\mathbf{H}^{(l - 1)})$, which is undefined since it's matrix-to-matrix. However, we can derive it without trouble with the Fr&#233;chet derivative as follows:

$$
\begin{align*}
    d(\mathcal{J})
    &= \operatorname{tr}(
        {\nabla_{\mathbf{H}^{(l)}}(\mathcal{J})}^\top
        d(\mathbf{H}^{(l)})
    ) \\
    &= \operatorname{tr}(
        {\nabla_{\mathbf{H}^{(l)}}(\mathcal{J})}^\top
        d(\phi(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)}))
    ) \\
    &= \operatorname{tr}(
        {\nabla_{\mathbf{H}^{(l)}}(\mathcal{J})}^\top
        (
            \phi'(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)})
            \odot d(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)})
        )
    ) \\
    &= \operatorname{tr}(
        (
            {\nabla_{\mathbf{H}^{(l)}}(\mathcal{J})}^\top
            \odot {\phi'(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)})}^\top
        )
        d(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)})
    ).
\end{align*}
$$

Here $\phi'(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)})$ is evaluated element-wise. To compute $\nabla_\mathcal{J}(\mathbf{W}^{(l)})$, we separate out $d(\mathbf{W}^{(l)})$ as follows:

$$
\begin{align*}
    & d(\mathcal{J}) = \operatorname{tr}(
        (
            {\nabla_{\mathbf{H}^{(l)}}(\mathcal{J})}^\top
            \odot {\phi'(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)})}^\top
        )
        \mathbf{H}^{(l - 1)}d(\mathbf{W}^{(l)})
    ) \\
    \Rightarrow
    & \nabla_\mathcal{J}(\mathbf{W}^{(l)}) = (
        (
            {\nabla_{\mathbf{H}^{(l)}}(\mathcal{J})}^\top
            \odot {\phi'(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)})}^\top
        )
        \mathbf{H}^{(l - 1)}
    )^\top \\
    \Rightarrow
    & \;\;\;\;\;\;\;\;\;\;\, = {\mathbf{H}^{(l - 1)}}^\top(
        \nabla_{\mathbf{H}^{(l)}}(\mathcal{J})
        \odot \phi'(\mathbf{H}^{(l - 1)}\mathbf{W}^{(l)})
    )
\end{align*}
$$